# URL Scrap

Scrap URL Today

In [16]:
from datetime import date
from datetime import datetime

today = date.today().strftime("%Y-%m-%d")

!echo =============== START =============== | tee -a "./logs/{today}.log"
!echo {datetime.now().strftime("%d-%b-%Y-%H:%M:%S")} [CRAWLER] [U] Starting URL scrap | tee -a "./logs/{today}.log"

!scrapy crawl url_spider -O ./url/daily/{today}.json 2>&1 3>&1 | tee -a "./logs/{today}.log"

!echo {datetime.now().strftime("%d-%b-%Y-%H:%M:%S")} [CRAWLER] [U] Finished URL scrap | tee -a "./logs/{today}.log"

=============== START ===============
22-May-2022-09:51:56 [CRAWLER] [U] Starting URL scrap
2022-05-22 09:51:58 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: tutorial)
2022-05-22 09:51:58 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 18.9.0, Python 3.8.10 (default, Mar 15 2022, 12:22:08) - [GCC 9.4.0], pyOpenSSL 19.0.0 (OpenSSL 1.1.1f  31 Mar 2020), cryptography 2.8, Platform Linux-5.13.0-1025-gcp-x86_64-with-glibc2.29
2022-05-22 09:51:58 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'tutorial',
 'DUPEFILTER_CLASS': 'scrapy.dupefilters.BaseDupeFilter',
 'LOG_LEVEL': 'INFO',
 'NEWSPIDER_MODULE': 'tutorial.spiders',
 'RETRY_HTTP_CODES': [429],
 'SPIDER_MODULES': ['tutorial.spiders']}
2022-05-22 09:51:58 [scrapy.extensions.telnet] INFO: Telnet Password: 74aab0c5362df190
2022-05-22 09:51:58 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telne

Merge Daily URL into Database

In [17]:
!echo =============== URLCB =============== | tee -a "./logs/{today}.log"

import json
import os

# Create empty JSONArray if it doesn't exist
url_db_file = open('./url/url_db.json', 'r' if os.path.exists('./url/url_db.json') else 'w+')
daily_file = open('./url/daily/' + today + '.json')

daily_json = json.load(daily_file)
url_db_json = json.load(url_db_file) if os.stat("./url/url_db.json").st_size > 2 else []

url_id_list = [item['id'] for item in url_db_json]

# Read daily file and append to url_db_json 
found = len(daily_json)
added = 0
for item in daily_json:
    if item['id'] not in url_id_list:
        url_db_json.append(item)
        url_id_list.append(item['id'])
        added = added + 1

!echo {datetime.now().strftime("%d-%b-%Y-%H:%M:%S")} [CRAWLER] [U] Total {len(url_db_json)}, Found {found}, added {added}, duplicated {found-added} URLs | tee -a "./logs/{today}.log"

# Write result file
with open('./url/url_db.json', "w") as url_db_file:
    url_db_file.write('[' +
            ',\n'.join(json.dumps(i) for i in url_db_json) +
            ']\n')

=============== URLCB ===============
22-May-2022-09:52:05 [CRAWLER] [U] Found 2052 URLs, added 187 URLs, duplicated 1865 URLs


# Scrap Projects

In [ ]:
from datetime import date
from datetime import datetime

!echo =============== PROJC =============== | tee -a "./logs/{today}.log"
!echo {datetime.now().strftime("%d-%b-%Y-%H:%M:%S")} [CRAWLER] [P] Starting project crawler | tee -a "./logs/{today}.log"

today = date.today().strftime("%Y-%m-%d")

!scrapy crawl project_spider -s JOBDIR=./crawls/project_spider/ -O ./projects/daily/{today}.json 2>&1 3>&1 | tee -a "./logs/{today}.log"

!echo {datetime.now().strftime("%d-%b-%Y-%H:%M:%S")} [CRAWLER] [P] Finished project crawler | tee -a "./logs/{today}.log"


=============== PROJC ===============
22-May-2022-09:52:05 [CRAWLER] [P] Starting project crawler
2022-05-22 09:52:06 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: tutorial)
2022-05-22 09:52:06 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 18.9.0, Python 3.8.10 (default, Mar 15 2022, 12:22:08) - [GCC 9.4.0], pyOpenSSL 19.0.0 (OpenSSL 1.1.1f  31 Mar 2020), cryptography 2.8, Platform Linux-5.13.0-1025-gcp-x86_64-with-glibc2.29
2022-05-22 09:52:06 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_MAX_DELAY': 5,
 'AUTOTHROTTLE_START_DELAY': 0.2,
 'AUTOTHROTTLE_TARGET_CONCURRENCY': 0.7,
 'BOT_NAME': 'tutorial',
 'DUPEFILTER_CLASS': 'scrapy.dupefilters.BaseDupeFilter',
 'LOG_LEVEL': 'INFO',
 'NEWSPIDER_MODULE': 'tutorial.spiders',
 'RETRY_HTTP_CODES': [429],
 'SPIDER_MODULES': ['tutorial.spiders']}
2022-05-22 09:52:06 [scrapy.extensions.telnet] INFO: Telnet Password: 34b4fe0517580798
2022-05-22 09:52:06 [s

# Upload to GDrive

In [3]:
!rm -rf ./crawls
import json
import os
from datetime import date
from datetime import datetime

today = date.today().strftime("%Y-%m-%d")

url_db_file = json.load(open('./url/url_db.json', 'r'))
daily_projects =json.load(open('./projects/daily/' + today + '.json'))

!echo {datetime.now().strftime("%d-%b-%Y-%H:%M:%S")} [CRAWLER] [G] Checking, crawled {len(daily_projects)} out of {len(url_db_file)} | tee -a "./logs/{today}.log"
#today = "2000-00-00"

!\cp {'./projects/daily/' + today + '.json'} ~/gdrive/tutorial/projects/daily/
!\cp {'./url/url_db.json'} ~/gdrive/tutorial/url/
!\cp {'./url/daily/' + today + '.json'} ~/gdrive/tutorial/url/daily/

!echo {datetime.now().strftime("%d-%b-%Y-%H:%M:%S")} [CRAWLER] [G] Uploaded to Google Drive | tee -a "./logs/{today}.log"
!echo ================ END ================ | tee -a "./logs/{today}.log"

23-May-2022-20:54:44 [CRAWLER] [G] Checking, crawled 7120 out of 7223
23-May-2022-20:55:00 [CRAWLER] [G] Uploaded to Google Drive
================ END ================
